In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import torch
# Simple hook function that just prints the output shape
def hook(module, input, output):
    print(f"Output shape: {output.shape}")

# Find and register hook to first attention block's output projection
for name, module in model.named_modules():
    if "self_attn.o_proj" in name and "layers.0" in name:
        print(f"Adding hook to: {name}")
        module.register_forward_hook(hook)



Adding hook to: model.layers.0.self_attn.o_proj


In [23]:
# Test it with some input
inputs = tokenizer("i have an apple", return_tensors="pt").to(model.device)
print(inputs['attention_mask'])
# with torch.no_grad():
#     outputs = model(**inputs)

tensor([[1, 1, 1, 1]], device='cuda:0')
